In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
# path to the ARC-MTQE directory
main_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(main_dir, "data")

In [3]:
def print_data_stats(n_rows, n_segments, n_critical_errors):
    """
    Print data summary statistics.
    """
    print(f"Number of rows: {n_rows}")
    print(f"Number of segments: {n_segments}")
    print(f"Number of critical errors: {n_critical_errors}")
    print(f"Percentage critical errors: {n_critical_errors/n_segments}")

## WMT 2021 critical errors

In [4]:
# contains train, dev and test data
mlqe_pe_data_dir = os.path.join(data_dir, "mlqe-pe", "data")

language_pairs = ["encs", "ende", "enja", "enzh"]

for lp in language_pairs:
    n_rows = 0
    n_segments = 0
    n_critical_errors = 0

    path_dev = os.path.join(mlqe_pe_data_dir, "catastrophic_errors", f"{lp}_majority_dev.tsv")
    path_train = os.path.join(mlqe_pe_data_dir, "catastrophic_errors", f"{lp}_majority_train.tsv")
    path_test = os.path.join(mlqe_pe_data_dir, "catastrophic_errors", f"{lp}_majority_test_blind.tsv")
    path_goldlabels = os.path.join(mlqe_pe_data_dir, "catastrophic_errors_goldlabels", f"{lp}_majority_test_goldlabels", "goldlabels.txt")

    df_dev = pd.read_csv(path_dev, sep="\t", header=None, names=["idx", "source", "target", "annotations", "label"])
    df_train = pd.read_csv(path_train, sep="\t", header=None, names=["idx", "source", "target", "annotations", "label"])
    df_test = pd.read_csv(path_test, sep="\t", header=None, names=["idx", "source", "target"])
    df_labels = pd.read_csv( path_goldlabels, sep="\t", header=None, names=["lang_pair", "ref", "idx", "label"])
    df_test_labelled = pd.merge(df_test, df_labels, on='idx')

    for df in [df_train, df_dev, df_test_labelled]:
        n_rows += df.shape[0]
        n_segments += df["idx"].nunique()
        n_critical_errors += df[df["label"] == "ERR"].shape[0]

    print(lp)
    print_data_stats(n_rows, n_segments, n_critical_errors)
    print("\n")


encs
Number of rows: 9476
Number of segments: 9476
Number of critical errors: 1637
Percentage critical errors: 0.17275221612494723


ende
Number of rows: 9878
Number of segments: 9878
Number of critical errors: 2773
Percentage critical errors: 0.2807248430856449


enja
Number of rows: 9658
Number of segments: 9658
Number of critical errors: 897
Percentage critical errors: 0.0928763719196521


enzh
Number of rows: 8859
Number of segments: 8859
Number of critical errors: 1409
Percentage critical errors: 0.1590472965345976




## WMT 2022 critical errors

In [5]:
wmt_data_dir = os.path.join(data_dir, "wmt-qe-2022-data")

language_pairs = ["en-de", "pt-en"]
for lp in language_pairs:
    n_rows = 0
    n_segments = 0
    n_critical_errors = 0

    path_train = os.path.join(wmt_data_dir, "train-dev_data", "task3_ced", "train", lp, f"{lp}-train", "train.label")
    path_dev = os.path.join(wmt_data_dir, "train-dev_data", "task3_ced", "dev", lp, f"{lp}-dev", "dev.label")
    path_test = os.path.join(wmt_data_dir, "test_data-gold_labels", "task3_ced", lp, f"test.2022.{lp}.label")

    df_train_labels = pd.read_csv(path_train, names=["label"])
    df_dev_labels = pd.read_csv(path_dev, names=["label"])
    df_test_labels = pd.read_csv(path_test, names=["label"])

    for df in [df_train_labels, df_dev_labels, df_test_labels]:
        n = df.shape[0]
        n_bad = df[df["label"]=="BAD"].shape[0]

        n_rows += n
        n_segments += n
        n_critical_errors += n_bad

    print(lp)
    print_data_stats(n_rows, n_segments, n_critical_errors)
    print("\n")

en-de
Number of rows: 173291
Number of segments: 173291
Number of critical errors: 9951
Percentage critical errors: 0.057423640004385684


pt-en
Number of rows: 44862
Number of segments: 44862
Number of critical errors: 2593
Percentage critical errors: 0.057799473942311975




## DEMETR

The below numbers do not correspond to the paper which lists 10 critical error categories. However, the dataset has 12 critical error categories + one of the baselines is listed as a critical error as well. This brings the total to 13.

In [6]:
demetr_data_dir = os.path.join(data_dir, "demetr", "dataset")
dfs = []
for filename in os.listdir(demetr_data_dir):
    f = os.path.join(demetr_data_dir, filename)
    df = pd.read_json(f)
    dfs.append(df)

demetr_df = pd.concat(dfs)

In [7]:
print("Number of segments per language pair: ", demetr_df.groupby('lang_tag')['id'].count().unique())
print("Number of unique segments per language pair: ", demetr_df.groupby('lang_tag')['id'].nunique().unique())
print("Number of error IDs per language pair: ", demetr_df.groupby('lang_tag')['pert_id'].nunique().unique())
print("Number of error names per language pair: ", demetr_df.groupby('lang_tag')['pert_name'].nunique().unique())
print("Number of language pairs per error category: ", demetr_df.groupby('pert_name')['lang_tag'].nunique().unique())

Number of segments per language pair:  [3500]
Number of unique segments per language pair:  [100]
Number of error IDs per language pair:  [33]
Number of error names per language pair:  [35]
Number of language pairs per error category:  [10]


The below cells show that `pert_id` values 5 and 6 are missing. Looking at the corresponding files `major_id5_pp_removed` and `critical_id6_addition.json`, the listed `pert_id` within the files is 8 instead of the expected 5 and 6. However, it seems that the `pert_name` column is used correctly in all files.

In [8]:
set(np.arange(1, 36, 1)) - set(demetr_df['pert_id'].unique())

{5, 6}

In [9]:
n_rows = n_segments = demetr_df.shape[0]
# n_critical_errors = demetr_df[demetr_df['severity'] == 'critical'].shape[0]

# note: in the paper, there are 10 critical error categories --> 10,000 segments 
n_critical_errors = 10000

print_data_stats(n_rows, n_segments, n_critical_errors)

Number of rows: 35000
Number of segments: 35000
Number of critical errors: 10000
Percentage critical errors: 0.2857142857142857


## Unbabel MQM

In [10]:
file_path = os.path.join(data_dir, "unbabel", "mqm_generalMT2022_enru.tsv")

df = pd.read_csv(file_path, sep="\t", index_col=False)
df[['seg_id', 'category', 'severity']].tail()

/var/folders/3h/4b2rm8kn7l9d6j583_k1mx1w0000gr/T/ipykernel_66044/373853115.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(file_path, sep="\t", index_col=False)


,seg_id,category,severity
33976,1553,omission,major
33977,1554,No-error,No-error
33978,1555,No-error,No-error
33979,1556,mistranslation,minor
33980,1557,mistranslation,major


In [11]:
df['crit_error'] = np.where(df["severity"] == "critical", 1, 0)

In [12]:
n_rows = df.shape[0]
n_segments = df['seg_id'].nunique()
n_critical_errors = sum(df.groupby('seg_id')['crit_error'].sum() >= 1)

print_data_stats(n_rows, n_segments, n_critical_errors)

Number of rows: 33981
Number of segments: 1215
Number of critical errors: 92
Percentage critical errors: 0.0757201646090535
